In [1]:
# imports
import datasets
import evaluate
import numpy as np
import peft
import pickle
import transformers

from datasets import(
    load_dataset, 
)


from peft import(
    LoraConfig,
    get_peft_model,
)

from transformers import(
AutoModelForCausalLM,
AutoTokenizer,
TrainingArguments
)
from trl import SFTTrainer, SFTConfig

/home/ac.cdavies/.vnv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-31 18:37:37.938287: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743446257.952604 1791354 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743446257.957034 1791354 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743446257.969697 1791354 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 

In [2]:
# model
def model_from_pkl(model):
    with open("pkl_files/" + model + ".pkl", "rb") as f:
        pkl_model=pickle.load(f)
    model_name=pkl_model["model_name"]
    tokenizer=pkl_model["tokenizer"]
    tokenizer.pad_token=tokenizer.eos_token
    return model_name, tokenizer
llama, llama_tokenizer=model_from_pkl("Llama-3.2-1B")

In [3]:
# LORA config
config=LoraConfig(
    #inference_mode=False,
    r=8, #rank of update matrices, lower value results in smaller matrices with fewer parameters
    lora_alpha=16, #LoRA scaling factor
    task_type="CAUSAL_LM",
    lora_dropout=0.1, # dropout probability of LoRA layers
    bias="none", # specifies if bias parameters should be trained
    #modules_to_save=["decode_head"] #models apart from LoRA layers that are trainable
)
lla_lora_model=get_peft_model(llama, config)

In [4]:
# data
ds_gst1_train=load_dataset("LongSafari/open-genome", "stage1", split="train[:500]")
#print(ds_gst1[50])
ds_gst1_test=load_dataset("LongSafari/open-genome", "stage1", split="test[:50]")
def tokenize_l_function(examples):
        return llama_tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_train=ds_gst1_train.map(tokenize_l_function, batched=True)
tokenized_test=ds_gst1_test.map(tokenize_l_function, batched=True)

In [5]:
# define trainer parameters (besides compute_metrics)
training_args=TrainingArguments(output_dir="test_trainer", 
                                #eval_strategy="epoch", 
                                per_device_train_batch_size=8,
                                num_train_epochs=3, # total number of training epochs to perform)
                                learning_rate=2e-4,
                                weight_decay=0.1,
                                logging_strategy="steps",
                                #logging_steps=2,
                                gradient_checkpointing=True)
# eval_strategy=epoch ensures that the evaluation metric will be reported at the end of each epoch
# this helps us monitor evaluation metrics during fine-tuning

In [6]:
# compute_metrics
metric=evaluate.load("accuracy")
print(metric)

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [7]:
comp_metrics_output=[]
def compute_metrics(eval_pred):
    logits=eval_pred.predictions
    refs=eval_pred.label_ids
    #print(logits)
    print(type(logits))
    #print(refs)
    print(type(refs))
    log_32=logits.astype(np.int32)
    log_32=np.concatenate(log_32).tolist()
    print(len(log_32))
    ref_32=refs.astype(np.int32)
    ref_32=np.concatenate(ref_32).tolist()
    print(len(ref_32))
    #predictions = np.argmax(logits, axis=-1)
    predictions = np.argmax(log_32, axis=-1)
    # print(predictions)
    #comp_metrics_output.append(metric.compute(predictions=predictions))
    #print(metric.compute(predictions=predictions))
    return metric.compute(predictions=predictions, references=ref_32)

In [8]:
l_trainer = SFTTrainer(
    model=lla_lora_model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    peft_config=config,
    args=SFTConfig(output_dir="test_trainer", 
                   eval_strategy="epoch",
                   logging_strategy="steps",
                   logging_steps=2, 
                  ),
    compute_metrics=compute_metrics
)

In [9]:
l_trainer.train() 

/home/ac.cdavies/.vnv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.645400,2.586096,0.000469
2,0.647300,2.582443,0.000469
3,0.651700,2.580906,0.000391


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
51200
51200


/home/ac.cdavies/.vnv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
51200
51200


/home/ac.cdavies/.vnv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/ac.cdavies/.vnv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
51200
51200


TrainOutput(global_step=48, training_loss=0.6495533337195715, metrics={'train_runtime': 1137.6355, 'train_samples_per_second': 1.319, 'train_steps_per_second': 0.042, 'total_flos': 8976366305280000.0, 'train_loss': 0.6495533337195715})